# **READ ABOUT:**

Implicit group recommendation techniques paper

**For Implicit Rating**

---



In [ ]:
#using jaccard - own algorithm
import itertools
def implicit_dissimilarity_jaccard_func(implicit_user_trainMatrix):

    
    Dissimilarity_mat_implicit=np.zeros((num_groups, num_items))

    for group_id in dataset.group_user_Dict:
        #print(group_id)
        group_members= dataset.group_user_Dict[group_id]
        num_members=len(group_members)
        for i in range(num_items):
            dissimilarity=0
            for m, n in itertools.combinations(range(num_members), 2):

              user1_items = implicit_user_trainMatrix[m]
              user2_items = implicit_user_trainMatrix[n]
              jaccard_similarity = np.sum(np.logical_and(user1_items, user2_items)) / np.sum(np.logical_or(user1_items, user2_items))

              dissimilarity = dissimilarity+ (1 - jaccard_similarity)
            Dissimilarity_mat_implicit[group_id][i]=(2*dissimilarity)/(num_members*(num_members-1) )
    return Dissimilarity_mat_implicit
    #print(Dissimilarity_mat_implicit)

In [ ]:
#using pearson coeff - own algorithm
from scipy.stats import pearsonr 

def implicit_dissimilarity_pearson_func(implicit_user_trainMatrix):

    
    Dissimilarity_mat_implicit_using_pearson=np.zeros((num_groups, num_items))

    for group_id in dataset.group_user_Dict:
        print(group_id)
        group_members= dataset.group_user_Dict[group_id]
        num_members=len(group_members)
        for i in range(num_items):
           dissimilarity = 0
           for m, n in itertools.combinations(range(num_members), 2):
               user1_items = implicit_user_trainMatrix[m]
               user2_items = implicit_user_trainMatrix[n]
               pearson_similarity, _ = pearsonr(user1_items, user2_items)
               dissimilarity += 1 - pearson_similarity
           Dissimilarity_mat_implicit_using_pearson[group_id][i]=(2*dissimilarity)/(num_members*(num_members-1) )
    return Dissimilarity_mat_implicit_using_pearson
    #print(Dissimilarity_mat_implicit)

In [ ]:
Dissimilarity_mat_implicit_using_pearson=implicit_dissimilarity_pearson_func(dataset.implicit_user_trainMatrix)

import pickle
# open a file in binary mode to write the pickled data
with open("Dissimilarity_mat_implicit_using_pearson.pkl", "wb") as f:
    # dump the object into the file
    pickle.dump(Dissimilarity_mat_implicit_using_pearson, f)

In [ ]:
#import pickle
#with open("Dissimilarity_mat_implicit_using_pearson.pkl", "rb") as f:
#    # load the pickled object from the file
#    Dissimilarity_mat_implicit_using_pearson = pickle.load(f)



**Implicit MF**

In [ ]:
class MF_implicit:

    def __init__(self, train_mat, test_mat, latent=5, lr=0.01, reg=0.01):
        self.train_mat = train_mat  # the training rating matrix of size (#user, #movie)
        self.test_mat = test_mat  # the training rating matrix of size (#user, #movie)
        
        self.latent = latent  # the latent dimension
        self.lr = lr  # learning rate
        self.reg = reg  # regularization weight, i.e., the lambda in the objective function
        
        self.num_user, self.num_movie = train_mat.shape
        
        self.sample_user, self.sample_movie = self.train_mat.nonzero()  # get the user-movie paris having ratings in train_mat
        self.num_sample = len(self.sample_user)  # the number of user-movie pairs having ratings in train_mat

        self.user_test_like = []
        for u in range(self.num_user):
            self.user_test_like.append(np.where(self.test_mat[u, :] > 0)[0])

        self.P = np.random.random((self.num_user, self.latent))  # latent factors for users, size (#user, self.latent), randomly initialized
        self.Q = np.random.random((self.num_movie, self.latent))  # latent factors for users, size (#movie, self.latent), randomly initialized


    def negative_sampling(self):
        negative_movie = np.random.choice(np.arange(self.num_movie), size=(len(self.sample_user)), replace=True)
        true_negative = self.train_mat[self.sample_user, negative_movie] == 0
        negative_user = self.sample_user[true_negative]
        negative_movie = negative_movie[true_negative]
        return np.concatenate([self.sample_user, negative_user]), np.concatenate([self.sample_movie, negative_movie])

    def train(self, epoch=20):
        """
        Goal: Write your code to train your matrix factorization model for epoch iterations in this function
        Input: epoch -- the number of training epoch 
        """
        
        for ep in range(epoch):
            """ 
            Write your code here to implement the training process for one epoch, 
            at the end of each epoch, run self.test() to evaluate current version of MF.
            """
            print("epoch={}".format(ep))
            negative_training_set=self.negative_sampling()
            perm = np.random.permutation(len(negative_training_set[0]))

            # use fancy indexing to select the elements in the shuffled order
            negative_training_set_0 = negative_training_set[0][perm]
            negative_training_set_1 = negative_training_set[1][perm]

            #print(negative_training_set)
            #print((self.P).shape)
            #print((self.Q).shape)
            #print((QT).shape)
            #np.random.shuffle(negative_training_set)
            #QT=self.Q.T
            
            for i in range(len(negative_training_set[0])):
                user=negative_training_set_0[i]
                item=negative_training_set_1[i]
              ## Concurrently updated
                self.P[user],self.Q[item]=self.P[user]-self.lr*(2*(self.P[user].dot(self.Q[item])-self.train_mat[user,item])*self.Q[item]
                    +2*self.reg*self.P[user]),self.Q[item]-self.lr*(2*((self.P[user].dot(self.Q[item])-self.train_mat[user,item])*self.P[user]
                    +2*self.reg*self.Q[item]))
              
            self.test()
            
        """
        End of your code for this function
        """
            
    def predict(self):
        """
        Write your code here to implement the prediction function, which generates the ranked lists of movies 
        by the trained MF for every user, store the result (named 'recommendation') in a numpy array of size (#user, 50), where entry (u, k) 
        represents the movie id that is ranked at position k in the recommendation list to user u. Return the 'recommendation' variable. 
        """
        prediction_mat = np.matmul(self.P, self.Q.T)
        recommendation = []
        for u in range(self.num_user):
            scores = prediction_mat[u]
            train_like = np.where(train_mat[u, :] > 0)[0]
            scores[train_like] = -9999
            top50_iid = np.argpartition(scores, -50)[-50:]
            top50_iid = top50_iid[np.argsort(scores[top50_iid])[-1::-1]]
            recommendation.append(top50_iid)
        recommendation = np.array(recommendation)
        return recommendation
        
        
        """
        End of your code for this function
        """
        
    
    def test(self):
        recommendation = self.predict()

        recalls = np.zeros(3)
        precisions = np.zeros(3)
        user_count = 0.

        for u in range(self.num_user):
            test_like = self.user_test_like[u]
            test_like_num = len(test_like)
            if test_like_num == 0:
                continue
            rec = recommendation[u, :]
            hits = np.zeros(3)
            for k in range(50):
                if rec[k] in test_like:
                    if k < 50:
                        hits[2] += 1
                        if k < 20:
                            hits[1] += 1
                            if k < 5:
                                hits[0] += 1
            recalls[0] += (hits[0] / test_like_num)
            recalls[1] += (hits[1] / test_like_num)
            recalls[2] += (hits[2] / test_like_num)
            precisions[0] += (hits[0] / 5.)
            precisions[1] += (hits[1] / 20.)
            precisions[2] += (hits[2] / 50.)
            user_count += 1

        recalls /= user_count
        precisions /= user_count


        print('recall@5\t[%.6f],\t||\t recall@20\t[%.6f],\t||\t recall@50\t[%.6f]' % (recalls[0], recalls[1], recalls[2]))
        print('precision@5\t[%.6f],\t||\t precision@20\t[%.6f],\t||\t precision@50\t[%.6f]' % (precisions[0], precisions[1], precisions[2]))
        print('')

**IMPLICIT RECOMMENDATION PERFORMANCE**

In [ ]:
def calculate_Precision_Recall_at_K(social_relation_output_implicit, implicit_group_trainMatrix ):
    #k=100
  Recall_at_k=np.zeros(num_groups)
  Precision_at_k=np.zeros(num_groups)
  for group_id in range(len(social_relation_output_implicit)):
      TP=0
      TP_FP=0
      TP_FN=np.sum(implicit_group_trainMatrix[group_id], axis=0)

      for item_id in social_relation_output_implicit[group_id]:
          if implicit_group_trainMatrix[group_id][item_id]==1:
              TP+=1
              TP_FP+=1
          else:
              TP_FP+=1
      Precision_at_k[group_id]=(TP/TP_FP)
      if TP_FN!=0:
          Recall_at_k[group_id]=(TP/TP_FN)
  return Precision_at_k, Recall_at_k

**IMPLICIT RECOMMENDATION ANALYSIS**


In [ ]:
import matplotlib.pyplot as plt
  
def Precision_at_k_plot(Precision_at_k ):
    # corresponding x axis values
    x = list(range(1, num_groups+1))

    #fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_xlabel('Group Id')
    ax.set_ylabel('Precision@100')
    #ax.plot(x, Precision_at_k, color='blue')
    ax.bar(x, Precision_at_k[0], color='blue') #Jaccard
    #ax.bar(x, Precision_at_k[1], color='red') #Pearson
    # function to show the plot
    plt.show()

In [ ]:
import matplotlib.pyplot as plt
  
def Recall_at_k_plot(Recall_at_k ):
    # corresponding x axis values
    x = list(range(1, num_groups+1))

    #fig, axs = plt.subplots(1, 2, sharey=True, tight_layout=True)
    fig = plt.figure(figsize=(20, 10))
    ax2 = fig.add_subplot(1, 1, 1)
    ax2.set_xlabel('Group Id')
    ax2.set_ylabel('Recall@100')
    ax2.bar(x, Recall_at_k[0], color='blue')
    #ax2.bar(x, Recall_at_k[1], color='red')
    plt.show()

# **IMPLICIT RECOMMENDATION ANALYSIS**

In [ ]:
#permuted pipeline creation
#IMPLICIT
w_1=0.8
w_2=0.2

#using Dis= Pearson/Jaccard Implicit
overall_group_item_output_mat=concensus_heuristic_function(social_descriptor_groupwise, expertise_in_group_members_mat, Dissimilarity_mat,  dataset.implicit_user_trainMatrix, w_1, w_2)

#Top k Ranking
social_relation_output_implicit=rank_top_k(overall_group_item_output_mat, 100)


Precision_at_k, Recall_at_k= calculate_Precision_Recall_at_K(social_relation_output_implicit, dataset.implicit_group_trainMatrix )

#print(Precision_at_k)
mean_Precision_at_k = sum(Precision_at_k) / len(Precision_at_k)
print("Mean of Precision@k={}".format(mean_Precision_at_k))

print()
#print(Recall_at_k)
mean_Recall_at_k = sum(Recall_at_k) / len(Recall_at_k)
print("Mean of Recall@k={}".format(mean_Recall_at_k))

##print(Precision_at_k)
#mean_Pearson_Precision_at_k = sum(Precision_at_k_Pearson) / len(Precision_at_k_Pearson)
#print("Mean of Pearson Precision@k={}".format(mean_Pearson_Precision_at_k))
#
#print()
##print(Recall_at_k)
#mean_Pearson_Recall_at_k = sum(Recall_at_k_Pearson) / len(Recall_at_k_Pearson)
#print("Mean of Pearson Recall@k={}".format(mean_Pearson_Recall_at_k))

In [ ]:
Recall_at_k=[]
Recall_at_k.append(Recall_at_k)
#Recall_at_k.append(Recall_at_k_Jaccard)
#Recall_at_k.append(Recall_at_k_Pearson)
Recall_at_k_plot(Recall_at_k )

In [ ]:
Precision_at_k=[]
Precision_at_k.append(Precision_at_k)
#Precision_at_k.append(Precision_at_k_Jaccard)
#Precision_at_k.append(Precision_at_k_Pearson)
Precision_at_k_plot(Precision_at_k )

# *b. IMPLICIT RECOMMENDATION*

**Assumption:** 

1.   K=100



\begin{array}{|c|c|c|c|c|} \hline
Social Relationship & Expertise Descriptor & Dissimilarity Descriptor &Precision@K & Recall@K \\ \hline
yes & yes & Jaccard & 23.71\% & 7.23\%\\ \hline
yes & yes & Pearson & 23.71\% & 7.23\%\\ \hline
foo & bar & foo & bar & 1  \\ \hline
\end{array}
